# Experiment : Network growth

## Init the Ray client

In [1]:
# Script for generate  experiments
# Imports
from clases.red_rddas_model import RedRddasModel
import ray

import time
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Ray Configurations
# runtime_env = {"working_dir": "/home/carlos/Documents/RESEARCH/SynEstRDDA", "pip": ["requests", "pendulum==2.1.2"]}
# ray.init(address='ray://172.17.163.244:10001', runtime_env=runtime_env , log_to_driver=False)

ray.init(log_to_driver=False, num_cpus=12)

RayContext(dashboard_url='', python_version='3.8.10', ray_version='1.13.0', ray_commit='e4ce38d001dbbe09cd21c497fedd03d692b2be3e', address_info={'node_ip_address': '172.17.163.244', 'raylet_ip_address': '172.17.163.244', 'redis_address': None, 'object_store_address': '/tmp/ray/session_2022-06-28_15-16-55_179806_8371/sockets/plasma_store', 'raylet_socket_name': '/tmp/ray/session_2022-06-28_15-16-55_179806_8371/sockets/raylet', 'webui_url': '', 'session_dir': '/tmp/ray/session_2022-06-28_15-16-55_179806_8371', 'metrics_export_port': 61446, 'gcs_address': '172.17.163.244:46833', 'address': '172.17.163.244:46833', 'node_id': '0311341e711bc5fd278f056f69941d57b53264b3d4995a4ba8ab260a'})

In [2]:
# Experiment for RDD variation
n_experiments = 10
l_experiments = []
for cont_experiment in range(1,n_experiments+1):
    print("============================")
    print("Experiment:", cont_experiment)
    print("============================")

    # Variable Parameters
    n_rddas_min = 3
    n_rddas_max = 10

    # Fixed Parameters
    n_of_variables_rdda = 5
    n_of_signals_rdda = 2
    n_exit_variables = 2
    n_clauses_function = 2
    type_network = "ALEATORY"
    # this name has to be unique
    unique_path = "21_01_2022"

    # List of Result for the Experiments
    l_res_sample = []

    v_n_network = 1
    for n_of_rdds in range(n_rddas_min, n_rddas_max + 1):
        print("Number of Network:", v_n_network)
        print("-------------------------------")

        # path = "files/" + unique_path + "_" + str(n_of_rdds) + "_" + str(n_of_variables_rdda) + "_" + str(n_of_signals_rdda) \
        #        + "_" + str(n_exit_variables) + "_" + str(n_clauses_function)

        # generate the RDDAs of the Network of RDDAs
        print("generating the Network of RDDAs ...")
        oRedRddasModel = RedRddasModel(n_of_rdds, n_of_variables_rdda, n_of_signals_rdda, n_exit_variables,
                                       n_clauses_function)

        # Generate the RDDs
        print("generating the rdds ...")
        oRedRddasModel.generate_rddas(type_network=type_network)

        # # Save the Network of RDDAs in a Pickle file
        # RedRddasModel.save_file_pickle(oRedRddasModel, path)
        # path += ".pickle"
        #
        # print("=======================================================")
        # print("The Network of RDDAs is saved in: ", path)

        # Show the Network of RDDAs
        # oRedRddasModel.show()

        # Calculate the Attractors by RDDA and by Signal
        v_begin_0 = time.time()
        result = RedRddasModel.find_attractors_rddas_ray.remote(oRedRddasModel)
        oRedRddasModel = ray.get(result)
        v_end_0 = time.time()
        v_time_0 = v_end_0 - v_begin_0

        # Calculate the Attractors by RDDA and by Signal
        v_begin_1 = time.time()
        result = RedRddasModel.calculation_compatible_pairs.remote(oRedRddasModel)
        oRedRddasModel = ray.get(result)
        v_end_1 = time.time()
        v_time_1 = v_end_1 - v_begin_1

        # # Calculate the Attractors by RDDA and by Signal with iterative Method
        # v_begin_1 = time.time()
        # # result = RedRddasModel.assembly_attractor_fields_iterative.remote(oRedRddasModel)
        # # oRedRddasModel = ray.get(result)
        # v_end_1 = time.time()
        # v_time_1 = v_end_1 - v_begin_1

        # Calculate the Attractors by RDDA and by Signal with optimized Method
        v_begin_2 = time.time()
        result = RedRddasModel.assembly_attractor_fields_optimized.remote(oRedRddasModel)
        oRedRddasModel = ray.get(result)
        v_end_2 = time.time()
        v_time_2 = v_end_2 - v_begin_2

        # Save the results for the experiment , numeric and time indicators
        res_dict = {
                    "n_network": v_n_network,
                    "n_rdds": n_of_rdds,
                    "n_rdda_attractors": len(oRedRddasModel.d_global_rdda_attractor.items()),
                    "t_find_attractors_method": v_time_0,
                    "n_pair_attractors": len(oRedRddasModel.list_attractors_pairs),
                    "t_comp_paris_method": v_time_1,
                    "n_attractor_fields": len(oRedRddasModel.attractor_fields),
                    "t_optimized_method": v_time_2
                    }
        l_res_sample.append(res_dict)
        v_n_network = v_n_network + 1

    # Add the sample data to pandas dataframe
    df = pd.DataFrame.from_dict(l_res_sample)
    l_experiments.append(df)
print("END EXPERIMENT")

Experiment: 1
Number of Network: 1
-------------------------------
generating the Network of RDDAs ...
generating the rdds ...
Generating the True Table
A ∨ B
{'7': 'A', '10': 'B'}
{'A': True, 'B': True}
A ∨ B
{'7': 'A', '10': 'B'}
{'A': True, 'B': False}
A ∨ B
{'7': 'A', '10': 'B'}
{'A': False, 'B': True}
A ∨ B
{'7': 'A', '10': 'B'}
{'A': False, 'B': False}
Generating the True Table
A ∨ B
{'15': 'A', '11': 'B'}
{'A': True, 'B': True}
A ∨ B
{'15': 'A', '11': 'B'}
{'A': True, 'B': False}
A ∨ B
{'15': 'A', '11': 'B'}
{'A': False, 'B': True}
A ∨ B
{'15': 'A', '11': 'B'}
{'A': False, 'B': False}
Generating the True Table
A ∨ B
{'5': 'A', '2': 'B'}
{'A': True, 'B': True}
A ∨ B
{'5': 'A', '2': 'B'}
{'A': True, 'B': False}
A ∨ B
{'5': 'A', '2': 'B'}
{'A': False, 'B': True}
A ∨ B
{'5': 'A', '2': 'B'}
{'A': False, 'B': False}
Generating the True Table
A ∨ B
{'2': 'A', '4': 'B'}
{'A': True, 'B': True}
A ∨ B
{'2': 'A', '4': 'B'}
{'A': True, 'B': False}
A ∨ B
{'2': 'A', '4': 'B'}
{'A': False, 'B':

In [3]:
# Experiment for RDD variation
n_experiments = 10
l_experiments = []
for cont_experiment in range(1,n_experiments+1):
    print("============================")
    print("Experiment:", cont_experiment)
    print("============================")

    # Variable Parameters
    n_rddas_min = 3
    n_rddas_max = 10

    # Fixed Parameters
    n_of_variables_rdda = 5
    n_of_signals_rdda = 2
    n_exit_variables = 2
    n_clauses_function = 2
    type_network = "ALEATORY"
    # this name has to be unique
    unique_path = "21_01_2022"

    # List of Result for the Experiments
    l_res_sample = []

    v_n_network = 1
    for n_of_rdds in range(n_rddas_min, n_rddas_max + 1):
        print("Number of Network:", v_n_network)
        print("-------------------------------")

        path = "files/" + unique_path + "_" + str(n_of_rdds) + "_" + str(n_of_variables_rdda) + "_" + str(n_of_signals_rdda) \
               + "_" + str(n_exit_variables) + "_" + str(n_clauses_function)

        # generate the RDDAs of the Network of RDDAs
        print("generating the Network of RDDAs ...")
        oRedRddasModel = RedRddasModel(n_of_rdds, n_of_variables_rdda, n_of_signals_rdda, n_exit_variables,
                                       n_clauses_function)

        # Generate the RDDs
        print("generating the rdds ...")
        oRedRddasModel.generate_rddas(type_network=type_network)

        # # Save the Network of RDDAs in a Pickle file
        # RedRddasModel.save_file_pickle(oRedRddasModel, path)
        # path += ".pickle"
        #
        # print("=======================================================")
        # print("The Network of RDDAs is saved in: ", path)

        # Show the Network of RDDAs
        # oRedRddasModel.show()

        # Calculate the Attractors by RDDA and by Signal
        v_begin_0 = time.time()
        result = RedRddasModel.find_attractors_rddas_ray.remote(oRedRddasModel)
        oRedRddasModel = ray.get(result)
        v_end_0 = time.time()
        v_time_0 = v_end_0 - v_begin_0

        # Calculate the Attractors by RDDA and by Signal
        v_begin_1 = time.time()
        result = RedRddasModel.calculation_compatible_pairs.remote(oRedRddasModel)
        oRedRddasModel = ray.get(result)
        v_end_1 = time.time()
        v_time_1 = v_end_1 - v_begin_1

        # # Calculate the Attractors by RDDA and by Signal with iterative Method
        # v_begin_1 = time.time()
        # # result = RedRddasModel.assembly_attractor_fields_iterative.remote(oRedRddasModel)
        # # oRedRddasModel = ray.get(result)
        # v_end_1 = time.time()
        # v_time_1 = v_end_1 - v_begin_1

        # Calculate the Attractors by RDDA and by Signal with optimized Method
        v_begin_2 = time.time()
        result = RedRddasModel.assembly_attractor_fields_optimized.remote(oRedRddasModel)
        oRedRddasModel = ray.get(result)
        v_end_2 = time.time()
        v_time_2 = v_end_2 - v_begin_2

        # Save the results for the experiment , numeric and time indicators
        res_dict = {
                    "n_network": v_n_network,
                    "n_rdds": n_of_rdds,
                    "n_rdda_attractors": len(oRedRddasModel.d_global_rdda_attractor.items()),
                    "t_find_attractors_method": v_time_0,
                    "n_pair_attractors": len(oRedRddasModel.list_attractors_pairs),
                    "t_comp_paris_method": v_time_1,
                    "n_attractor_fields": len(oRedRddasModel.attractor_fields),
                    "t_optimized_method": v_time_2
                    }
        l_res_sample.append(res_dict)
        v_n_network = v_n_network + 1

    # Add the sample data to pandas dataframe
    df = pd.DataFrame.from_dict(l_res_sample)
    l_experiments.append(df)
print("END EXPERIMENT")

Experiment: 1
Number of Network: 1
-------------------------------
generating the Network of RDDAs ...
generating the rdds ...
Generating the True Table
A ∨ B
{'9': 'A', '10': 'B'}
{'A': True, 'B': True}
A ∨ B
{'9': 'A', '10': 'B'}
{'A': True, 'B': False}
A ∨ B
{'9': 'A', '10': 'B'}
{'A': False, 'B': True}
A ∨ B
{'9': 'A', '10': 'B'}
{'A': False, 'B': False}
Generating the True Table
A ∨ B
{'11': 'A', '13': 'B'}
{'A': True, 'B': True}
A ∨ B
{'11': 'A', '13': 'B'}
{'A': True, 'B': False}
A ∨ B
{'11': 'A', '13': 'B'}
{'A': False, 'B': True}
A ∨ B
{'11': 'A', '13': 'B'}
{'A': False, 'B': False}
Generating the True Table
A ∨ B
{'14': 'A', '11': 'B'}
{'A': True, 'B': True}
A ∨ B
{'14': 'A', '11': 'B'}
{'A': True, 'B': False}
A ∨ B
{'14': 'A', '11': 'B'}
{'A': False, 'B': True}
A ∨ B
{'14': 'A', '11': 'B'}
{'A': False, 'B': False}
Generating the True Table
A ∨ B
{'4': 'A', '2': 'B'}
{'A': True, 'B': True}
A ∨ B
{'4': 'A', '2': 'B'}
{'A': True, 'B': False}
A ∨ B
{'4': 'A', '2': 'B'}
{'A': Fal

## Save the experiment data in csv, using pandas Dataframe

In [43]:
pf_res = pd.concat(l_experiments, keys=range(1,n_experiments+1), names=["n_sample","n_aux"], ignore_index=False)
pf_res.reset_index(drop=True, inplace=True, level=1)
pf_res

,n_network,n_rdds,n_rdda_attractors,t_find_attractors_method,n_pair_attractors,t_comp_paris_method,n_attractor_fields,t_optimized_method
n_sample,,,,,,,,
1,1,3,29,0.565986,264,0.062490,11,0.041013
1,2,4,40,0.586503,211,0.054445,139,0.051982
1,3,5,33,0.710711,116,0.037357,40,0.008785
1,4,6,31,0.706664,124,0.038347,23,0.081930
1,5,7,69,1.120260,522,0.110559,0,0.021936
...,...,...,...,...,...,...,...,...
10,4,6,71,1.101700,426,0.121982,336,1.060408
10,5,7,52,0.958010,188,0.049830,0,0.007257
10,6,8,55,0.994895,239,0.062933,0,0.004036


In [44]:
path = "exp1_network_growth_data.csv"
pf_res.to_csv(path)
print("Experiment saved in:", path)

Experiment saved in: exp1_network_growth_data.csv
